In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix 
import scipy.io as scio
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time
import pandas as pd
import scipy.io as scio
from scipy.stats import norm

In [10]:
class Dataset:
    
    def __init__(self,name,form):
        
        self.name=name
        self.form=form
        
    def load(self):
        
        self.data_dict=scio.loadmat(self.name+self.form)
        self.data=self.data_dict[self.name]

In [11]:
def get_wts(arr):
    return norm.cdf(arr,np.mean(arr),np.std(arr))

bots=Dataset("Botswana",".mat")
bots.load()
hypd=bots.data


shp=np.shape(hypd)
row=shp[0]
col=shp[1]
bands=shp[2]
    
hypd=hypd.reshape(row*col,bands)
    
bots_gt=Dataset('Botswana_gt','.mat')
bots_gt.load()
y=bots_gt.data.reshape(row*col,1)
ind=np.where(y[:,0]!=0)
X_hypd=hypd[ind]
y = y[y!=0]

In [12]:
df = pd.DataFrame(X_hypd)

df2=pd.DataFrame()

band=dict()
band['Blue']=np.linspace(490,530,5)
band['Green']=np.linspace(530,590,7)
band['Red']=np.linspace(630,680,6)
band['NIR']=np.linspace(850,880,4)



wblue=np.ediff1d(get_wts(band['Blue']))
wgreen=np.ediff1d(get_wts(band['Green']))
wred=np.ediff1d(get_wts(band['Red']))
wnir=np.ediff1d(get_wts(band['NIR']))


df2['Blue'] = (df.iloc[:, 0:4].multiply(wblue, axis=1).sum(axis=1))/np.sum(wblue)
df2['Green'] = (df.iloc[:, 4:10].multiply(wgreen, axis=1).sum(axis=1))/np.sum(wgreen)
df2['Red'] = (df.iloc[:, 14:19].multiply(wred, axis=1).sum(axis=1))/np.sum(wred)
df2['NIR'] = (df.iloc[:, 36:39].multiply(wnir, axis=1).sum(axis=1))/np.sum(wnir)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df2, y, test_size=0.33)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()  
sc.fit_transform(X_train)
Xn_train = sc.transform(X_train)
Xn_test = sc.transform(X_test)

In [14]:
mu=dict()
icov=dict()
det=dict()
start_time = time.time()
for cls in np.unique(y_train):
    ind=np.where(y_train==cls)
    X_cls=Xn_train[ind]
    mu[cls]=np.mean(X_cls,0)
    cov=np.cov(X_cls.T)
    det[cls]=np.linalg.det(cov)
    icov[cls] = np.linalg.inv(cov)
print("Training time is %s seconds ---" % (time.time() - start_time))

Training time is 0.2791762351989746 seconds ---


In [15]:
def find_class(test,mu,det,icov):
    tot_cls=len(det)
    pmax=-1000000000000000
    for i in list(range(1,15,1)):
        test_s=test-mu[i]
        logp=-np.log(det[i])-np.matmul(np.matmul(test_s,icov[i]),test_s.T)
    
        if(logp>pmax):
            pmax=logp
            y_pred=i
    return y_pred
    